In [2]:
!pip install awscli --force-reinstall --upgrade

^C


In [4]:
!pip install sagemaker

  Using cached attrs-23.2.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached numpy-1.26.4-cp39-cp39-win_amd64.whl.metadata (61 kB)
  Using cached botocore-1.35.99-py3-none-any.whl.metadata (5.7 kB)
  Using cached mock-4.0.3-py3-none-any.whl.metadata (2.8 kB)
  Using cached dill-0.3.9-py3-none-any.whl.metadata (10 kB)
  Using cached multiprocess-0.70.17-py39-none-any.whl.metadata (7.2 kB)
Using cached attrs-23.2.0-py3-none-any.whl (60 kB)
Using cached numpy-1.26.4-cp39-cp39-win_amd64.whl (15.8 MB)
Using cached botocore-1.35.99-py3-none-any.whl (13.3 MB)
Using cached dill-0.3.9-py3-none-any.whl (119 kB)
Using cached mock-4.0.3-py3-none-any.whl (28 kB)
Using cached multiprocess-0.70.17-py39-none-any.whl (133 kB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: dill
    Found existing installation: dill 0.3.8
    Uninstalling dill-0.3.8:
      Successfully uninstalled

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
doccano 1.8.4 requires chardet<5.0.0,>=4.0.0, which is not installed.
doccano 1.8.4 requires SQLAlchemy<2.0.0,>=1.4.31, which is not installed.
matplotlib 3.8.3 requires cycler>=0.10, which is not installed.
matplotlib 3.8.3 requires fonttools>=4.22.0, which is not installed.
matplotlib 3.8.3 requires kiwisolver>=1.3.1, which is not installed.
matplotlib 3.8.3 requires pillow>=8, which is not installed.
matplotlib 3.8.3 requires pyparsing>=2.3.1, which is not installed.
mlflow 2.17.1 requires Flask<4, which is not installed.
mlflow 2.17.1 requires markdown<4,>=3.3, which is not installed.
mlflow 2.17.1 requires sqlalchemy<3,>=1.4.0, which is not installed.
poetry 1.5.1 requires pexpect<5.0.0,>=4.7.0, which is not installed.
ray 2.4.0 requires grpcio<=1.51.3,>=1.32.0; python_version < "3.10" and sys_platform != "da

In [3]:
!pip install boto3 

^C



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


#### First attempt of ChartQA based quantization to AWQ

In [37]:
role = "YOUR_AWS_ROLE"
bucket_name = "llama-training-s3"
s3_prefix = "llama-training-s3/quantized"
s3_output_path=f"s3://{bucket_name}/{s3_prefix}/model"
hf_token="YOUR_HF_TOKEN"

In [ ]:
%mkdir training_code

In [34]:
%%writefile training_code/requirements.txt
boto3
transformers
torch
datasets
accelerate
sentencepiece
bitsandbytes
peft
pyarrow
deepspeed
accelerate
autoawq

Overwriting training_code/requirements.txt


In [65]:
%%writefile training_code/quantize_llm.py

import subprocess
import os
import torch
import argparse
from transformers import AutoTokenizer
import boto3
import sys


if not os.path.exists("AutoAWQ"):
    subprocess.run(["git", "clone", "https://github.com/casper-hansen/AutoAWQ.git"], check=True)

# Install AutoAWQ from the cloned repository in editable mode (this runs setup.py)
# Use an absolute path to ensure correct installation
autoawq_path = os.path.join(os.getcwd(), "AutoAWQ")
subprocess.run(["pip", "install", "-e", autoawq_path], check=True)

# Add the cloned repository to PYTHONPATH manually in case it's not discovered
sys.path.insert(0, autoawq_path)

# Install additional dependencies
subprocess.run(["pip", "install", "boto3"], check=True)
subprocess.run(["pip", "install", "--upgrade", "--force-reinstall", "transformers==4.50.3"], check=True)


role = "YOUR_AWS_ROLE"
bucket_name = "llama-training-s3"
s3_prefix = "llama-training-s3/quantized"


import json
import torch.nn as nn
from datasets import load_dataset
from awq import AutoAWQForCausalLM
from awq.utils.qwen_vl_utils import process_vision_info
from awq.quantize.quantizer import AwqQuantizer, clear_memory, get_best_device



# Parse arguments (S3 bucket and model path)
parser = argparse.ArgumentParser()
parser.add_argument("--model_path", type=str, default="javiagu/Qwen-2.5-ChartQA")
parser.add_argument("--quant_path", type=str, default="/opt/ml/model/Qwen-2.5-ChartQA")
parser.add_argument("--s3_output", type=str, required=True, help="S3 path to upload quantized model")
parser.add_argument("--hf_token", type=str, required=True, help="Hugging Face authentication token")

args = parser.parse_args()


# Specify paths and hyperparameters for quantization
quant_config = {"zero_point": True, "q_group_size": 128, "w_bit": 4, "version": "GEMM"}

# Load the model
model = AutoAWQForCausalLM.from_pretrained(
    args.model_path, attn_implementation="flash_attention_2"
)

model.to('cuda')

# Ensure the tokenizer uses left padding
model.processor.tokenizer.padding_side = 'left'

# Custom quantizer definition remains the same
class Qwen2VLAwqQuantizer(AwqQuantizer):
    def init_quant(self, n_samples=None, max_seq_len=None):
        modules = self.awq_model.get_model_layers(self.model)
        print("Number of layers to calibrate:", len(modules))
        print("First module before patching:", modules[0])

        samples = self.calib_data

        inps = []
        layer_kwargs = {}

        best_device = get_best_device()
        modules[0] = modules[0].to(best_device)
        self.awq_model.move_embed(self.model, best_device)

        # Patch layer 0 to capture input and kwargs.
        class Catcher(nn.Module):
            def __init__(self, module):
                super().__init__()
                self.module = module

            def forward(self, *args, **kwargs):
                # Assume first input to forward is hidden states.
                ### Debugging
                print("------------------")
                print("Catcher triggered!")
                print("------------------")
                ### Debugging End
                
                if len(args) > 0:
                    hidden_states = args[0]
                else:
                    first_key = list(kwargs.keys())[0]
                    hidden_states = kwargs.pop(first_key)
                inps.append(hidden_states)
                layer_kwargs.update(kwargs)
                raise ValueError  # early exit to break later inference

        def move_to_device(obj, device):
            def get_device(obj):
                if isinstance(obj, torch.Tensor):
                    return obj.device
                return next(obj.parameters()).device
            if get_device(obj) != device:
                obj = obj.to(device)
            return obj

        modules[0] = Catcher(modules[0])
        for k, v in samples.items():
            if isinstance(v, (torch.Tensor, nn.Module)):
                samples[k] = move_to_device(v, best_device)
        try:
            self.model(**samples)
        except ValueError:
            pass
        finally:
            for k, v in samples.items():
                if isinstance(v, (torch.Tensor, nn.Module)):
                    samples[k] = move_to_device(v, "cpu")
        modules[0] = modules[0].module  # restore original module

        del samples
        inps = inps[0]

        modules[0] = modules[0].cpu()
        self.awq_model.move_embed(self.model, "cpu")

        clear_memory()

        return modules, layer_kwargs, inps

# ----------------- Prepare Calibration Data -----------------
# Load and format the ChartQA dataset (or your own dataset)
dataset_id = "HuggingFaceM4/ChartQA"
raw_eval_dataset = load_dataset(dataset_id, split="val[:1%]")

system_message = (
    "You are a Vision Language Model specialized in interpreting visual data from chart images.\n"
    "Your task is to analyze the provided chart image and respond to queries with concise answers, "
    "usually a single word, number, or short phrase.\n"
    "The charts include a variety of types (e.g., line charts, bar charts) and contain colors, labels, and text.\n"
    "Focus on delivering accurate, succinct answers based on the visual information. Avoid additional explanation unless absolutely necessary."
)

def format_data(sample):
    label_text = sample["label"][0] if isinstance(sample["label"], list) else sample["label"]
    return [
        {
            "role": "system",
            "content": [{"type": "text", "text": system_message}],
        },
        {
            "role": "user",
            "content": [
                {"type": "image", "image": sample["image"]},
                {"type": "text", "text": sample["query"]},
            ],
        },
        {
            "role": "assistant",
            "content": [{"type": "text", "text": label_text}],
        },
    ]

# Format the evaluation dataset for calibration.
calib_dataset = [format_data(sample) for sample in raw_eval_dataset]

print("Total calibration samples:", len(calib_dataset))
print("First calibration sample:")
print(calib_dataset[0])


# Process the dataset into tensors for calibration.
# Note: Unlike training, we only need the inputs, not labels.
text = model.processor.apply_chat_template(calib_dataset, tokenize=False, add_generation_prompt=True)
image_inputs, video_inputs = process_vision_info(calib_dataset)
calib_inputs = model.processor(
    text=text, images=image_inputs, videos=video_inputs, padding=True, return_tensors="pt"
)

calib_inputs = {key: tensor.to("cuda") for key, tensor in calib_inputs.items()}


### Debugging
print("------------------------------")
print("Checking Calibration Content: ")
print("------------------------------")
# After obtaining calib_inputs
print("Calibration Data Summary:")
for key, tensor in calib_inputs.items():
    # Move tensor to CPU and convert to list for safe printing (if needed)
    tensor_cpu = tensor.detach().cpu()
    print(f"{key}: shape={tensor_cpu.shape}")
    # Print the first 5 elements of the tensor (adjust as needed)
    # This assumes the tensor is at least 1-dimensional.
    try:
        # If it's a multi-dimensional tensor, you can print the first sample.
        print(f"  Sample data: {tensor_cpu[0].tolist()[:10]}")  
    except Exception as e:
        print(f"  Could not print sample data: {e}")
### Debugging End

### Debugging
#try:
#    with torch.no_grad():
#        output = model(**calib_inputs)
#        print("Forward pass completed, output shape:", output.shape)
#except Exception as e:
#    print("Forward pass error:", e)
### Debugging End

# Perform quantization
print("Quantizing model...")
model.quantize(calib_data=calib_inputs, quant_config=quant_config, quantizer_cls=Qwen2VLAwqQuantizer)

# Enable cache before saving the quantized model.
model.model.config.use_cache = model.model.generation_config.use_cache = True
model.save_quantized(args.quant_path, safetensors=True, shard_size="4GB")

# Save quantized model locally
print(f"Saving quantized model at {args.quant_path}...")
model.save_quantized(args.quant_path)

tokenizer = model.processor.tokenizer
tokenizer.save_pretrained(args.quant_path)

# Upload to S3 for downloading later
print(f"Uploading quantized model to {args.s3_output}...")
s3_client = boto3.client("s3")
bucket_name, prefix = args.s3_output.replace("s3://", "").split("/", 1)

for root, _, files in os.walk(args.quant_path):
    for file in files:
        local_path = os.path.join(root, file)
        s3_path = os.path.join(prefix, os.path.relpath(local_path, args.quant_path))
        print(f"Uploading {local_path} to s3://{bucket_name}/{s3_prefix}")
        s3_client.upload_file(local_path, bucket_name, s3_prefix)

print("Quantization complete and uploaded to S3.")


Overwriting training_code/quantize_llm.py


In [ ]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    entry_point="quantize_llm.py",
    source_dir="training_code",
    role=role,
    instance_count=1,
    instance_type="ml.p4de.24xlarge",#"ml.g5.4xlarge",  # Change based on needs
    framework_version="2.5.1",
    py_version="py311",
    dependencies=["requirements.txt"],
    hyperparameters={
        "s3_output": s3_output_path,
        "hf_token": hf_token  # Add your Hugging Face token here
    },
    output_path=s3_output_path
)

# Fit the model
estimator.fit()


In [ ]:
!aws sagemaker stop-training-job --training-job-name pytorch-training-2025-04-01-06-21-06-319